# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [7]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

print("number of files are {0}".format(len(file_path_list)))
print(file_path_list[0])

/home/workspace
number of files are 30
/home/workspace/event_data/2018-11-27-events.csv


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [9]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("Total number of rows is {0}".format(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print("Example of the first 3 data rows: \n{0}".format(full_data_rows_list[:3]))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows is 8056
Example of the first 3 data rows: 
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88']]


In [10]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

-----

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [11]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [17]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [18]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [19]:
table_1 = 'artist_songs'
table_2 = 'artist_data'
table_3 = 'user_song'

In [20]:
def drop_table(session, tbl):
    """
    This function drops the table from the database.
    param session: database connection object
    param tbl: table name
    """
    session.execute("DROP TABLE IF EXISTS sparkify."+tbl+";")
    print("Table ( {0} ) has been deleted successfully!".format(tbl))

For _table1_ **artist_songs**, the columns **sessionId** and **itemInSession** were used as a composite partition keys because the requested query will be filtered by these columns.

In [28]:
# Query 1 target: get artist, song, length where sessionId = 338, and itemInSession = 4

query_1 = "CREATE TABLE "+table_1+" (sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY (sessionId, itemInSession));"
try:
    drop_table(session, table_1)
    session.execute(query_1)
except Exception as e:
    print(e)

Table ( artist_songs ) has been deleted successfully!


In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO "+table_1+" (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [30]:
# Query 1: get artist, song, length where sessionId = 338, and itemInSession = 4
query = "select artist, song, length from "+table_1+" WHERE sessionId = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:    print (row.artist, '|', row.song, '|', row.length)

Faithless | Music Matters (Mark Knight Dub) | 495.3073


----

For _table2_ **artist_data**, the columns **userId** and **sessionId** were used as a composite partition keys because the requested query will be filtered by these columns. Also, a composite partition key of **userId** and **sessionId** is used because it will increase the query performance since the data will be partiotioned based on this composite key. **itemInSession** was used as clustering column in order to sort the query result by this column in an ascending order.

In [31]:
# get artist, song (sorted by itemInSession), user (first and last name) where userid = 10, sessionid = 182
query_2 = "CREATE TABLE "+table_2+" (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession));"
try:
    drop_table(session, table_2)
    session.execute(query_2)
except Exception as e:
    print(e)    

Table ( artist_data ) has been deleted successfully!


In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO "+table_2+" (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [33]:
# Query 2: get artist, song (sorted by itemInSession), user (first and last name) where userid = 10, sessionid = 182
query = "select artist, song, firstName, lastName from "+table_2+" WHERE userId = 10 AND sessionId = 182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.artist, '|', row.song, '|', row.firstname, row.lastname)
                    

Down To The Bone | Keep On Keepin' On | Sylvie Cruz
Three Drives | Greece 2000 | Sylvie Cruz
Sebastien Tellier | Kilometer | Sylvie Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz


----

For _table3_ **user_song**, the column **song** was used as a partition keys because the requested query will be filtered by this column. Column **userId** was used as clustering column in order to help making up the unique key.

In [34]:
# Query 3: get firstName, lastName where song = 'All Hands Against His Own'

query_3 = "CREATE TABLE "+table_3+" (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId));"
try:
    drop_table(session, table_3)
    session.execute(query_3)
except Exception as e:
    print(e) 


Table ( user_song ) has been deleted successfully!


In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO "+table_3+" (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [36]:
query = "select firstName, lastName from "+table_3+" WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


----

### Drop the tables before closing out the sessions

In [37]:
drop_table(session, table_1)
drop_table(session, table_2)
drop_table(session, table_3)

Table ( artist_songs ) has been deleted successfully!
Table ( artist_data ) has been deleted successfully!
Table ( user_song ) has been deleted successfully!


### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()